<a href="https://colab.research.google.com/github/hoadoan1997/ChatGPT_App/blob/main/mdcs_binary_to_graph_ilp_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Required Libraries

In [ ]:
!pip install pulp

In [ ]:
import pandas as pd  # For handling tabular data (DataFrames)
import numpy as np  # For numerical operations and arrays
import time  # To measure execution time
import os  # For interacting with the operating system (e.g., file paths)
import networkx as nx  # For working with graphs and networks

from itertools import combinations  # To generate combinations of elements

# PuLP - Linear Programming library
from pulp import *
from pulp import LpVariable, LpProblem, LpMinimize, value, LpStatus, LpBinary  # Optimization components

# Convert Binary Dataset to Graph Format

In [2]:
start_time = time.time() # Start measuring execution time

# Load the binary training dataset (each feature is 0 or 1)
df = pd.read_csv("/content/binary_dataset/your_file.csv")

# Generate feature IDs starting after the last row ID
feature_start_id = len(df) + 1
feature_ids = {col: idx for idx, col in enumerate(df.columns, start=feature_start_id)}

output_lines = []
# Iterate through each row and collect feature links for value = 1
for idx, row in df.iterrows():
    transaction_id = idx + 1
    for col, val in row.items():
        if val == 1:
            feature_id = feature_ids[col]
            output_lines.append(f"{transaction_id} {feature_id}")

# Write the graph edges (transaction–feature) to a text file
with open("/content/graph_dataset/ccfraud_data.txt", "w") as f:
    f.write("\n".join(output_lines))

# Write the mapping of feature ID to feature name
with open("/content/feature_file/ccfraud_feature.txt", "w") as f:
    for col, fid in feature_ids.items():
        f.write(f"{fid} {col}\n")

end_time = time.time()
elapsed_time = end_time - start_time

print("File export successful.")
print(f"Execution time:: {elapsed_time:.2f} seconds")

Đã xuất file twitterbot_data.txt và twitterbot_feature.txt thành công.
Thời gian thực thi: 0.00 giây


# Function to Convert All Binary CSVs to Graph Format

In [1]:
# 🛠️ Function to Convert All Binary CSVs to Graph Format
def convert_all_binary_csv(input_dir, graph_output_dir, feature_output_dir):
    start_time = time.time()

    # Create output directories if they don't exist
    os.makedirs(graph_output_dir, exist_ok=True)
    os.makedirs(feature_output_dir, exist_ok=True)

    # Loop through all CSV files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith("_train_binary.csv"):
            print(f"🔄 Processing: {filename}")

            file_path = os.path.join(input_dir, filename)
            df = pd.read_csv(file_path)

            # Assign feature IDs (starting after number of rows)
            feature_start_id = len(df) + 1
            feature_ids = {col: idx for idx, col in enumerate(df.columns, start=feature_start_id)}

            output_lines = []

            # Generate transaction-feature edges where value == 1
            for idx, row in df.iterrows():
                transaction_id = idx + 1
                for col, val in row.items():
                    if val == 1:
                        feature_id = feature_ids[col]
                        output_lines.append(f"{transaction_id} {feature_id}")

            # Write graph file (edges)
            graph_file_path = os.path.join(
                graph_output_dir, filename.replace("_train_binary.csv", "_data.txt")
            )
            with open(graph_file_path, "w") as f:
                f.write("\n".join(output_lines))

            # Write feature mapping file
            feature_file_path = os.path.join(
                feature_output_dir, filename.replace("_train_binary.csv", "_feature.txt")
            )
            with open(feature_file_path, "w") as f:
                for col, fid in feature_ids.items():
                    f.write(f"{fid} {col}\n")

    elapsed = time.time() - start_time
    print(f"\n✅ Finished processing all datasets. Total time: {elapsed:.2f} seconds.")

# Run: Convert All Binary CSVs to Graph Format (for Google Colab)

In [2]:
# ▶️ Run: Convert All Binary CSVs to Graph Format (for Google Colab)

# Make sure you've uploaded the folder `binary_dataset` to /content/ in Colab
convert_all_binary_csv(
    input_dir="/content/binary_dataset", # Folder containing binary CSV files
    graph_output_dir="/content/graph_dataset", # Output: graph edge files
    feature_output_dir="/content/feature_file" # Output: feature ID mapping files
)

🔄 Đang xử lý: ccfraud_train_binary.csv
🔄 Đang xử lý: twitterbot_train_binary.csv
🔄 Đang xử lý: fraudecom_train_binary.csv
🔄 Đang xử lý: ipblock_train_binary.csv
🔄 Đang xử lý: fakejob_train_binary.csv
🔄 Đang xử lý: vehicleloan_train_binary.csv
🔄 Đang xử lý: malurl_train_binary.csv

✅ Xử lý hoàn tất toàn bộ dataset. Thời gian: 4.85 giây.


# Solve Minimum Discriminating Code Set (MDCS) via Integer Linear Programming

In [ ]:
# To read edge-lists stored as txt files
def genGraph():
    G = nx.Graph()
    G = nx.read_edgelist("/content/graph_dataset/your_file.txt", nodetype=int)
    return G


# This function computes the optimal MDCS for the input graph
def model():
    # Inputs the total number of nodes in the graph
    numNodes = int(input("Enter the number of nodes: "))

    # Inputs the nodes to be uniquely monitored in the graph
    transformers = int(input("Enter the number of transformers: "))

    start = time.time()

    # Prepare node groups
    nodes = [i + 1 for i in range(transformers, numNodes)] # Nodes to be uniquely monitored
    tnodes = [i + 1 for i in range(transformers)]          # Remaining nodes (can be selected as resources)
    totNodes = tnodes + nodes

    G = genGraph()
    G.add_nodes_from(totNodes) # Add all possible nodes to the graph

    print("Initializing Integer Linear Program")
    print("-----------------------------------")
    problem = LpProblem("IdentifyingCodes1", LpMinimize)

    # Create binary variables x_i for each node
    x = LpVariable.dict("x_%s", totNodes, 0, 1, LpBinary)

    # Objective: minimize the number of resource nodes used
    problem += sum(x[i] for i in nodes)

    # Coloring Constraint: every monitored node must be covered by at least one neighbor
    print("Adding Coloring Constraints")
    print("-----------------------------------")
    for i in tnodes:
        valColor = 0 # Initialize valColor outside the inner loop
        neighbor = list(G.neighbors(i))
        #print(i)
        #print(list(G.neighbors(i)))
        #print(neighbor)
        for j in neighbor:
            valColor += x[j]
        # Ensure the constraint is valid before adding
        if valColor >= 1:
            problem += valColor >= 1, "Coloring_Constraint_{}".format(i)
    valUnique = 0 # Initialize valUnique outside the inner loop
    neighbor_i = []
    neighbor_j = []

    # Uniqueness Constraint: each pair of monitored nodes must have different monitoring sets
    print("Adding Uniqueness Constraints")
    print("-----------------------------------")
    comb = combinations(tnodes, 2)
    for i in comb:
        pair = list(i)
        #print(pair)
        node1 = pair[0]
        node2 = pair[1]
        neighbor1 = list(G.neighbors(node1))
        neighbor2 = list(G.neighbors(node2))
        set1 = set(neighbor1)
        set2 = set(neighbor2)
        unique = list(set1.symmetric_difference(set2))
        #print(unique)
        valUnique = 0 # Initialize valUnique for each pair of nodes
        for k in unique:
            valUnique += x[k]
        # Ensure the constraint is valid before adding
        if valUnique >= 1:
            problem += valUnique >= 1, "Uniqueness_Constraint_{}".format(i)

    # Solve the ILP
    print("Solving")
    print("-------------------------------------------------------")
    problem.solve()

    # Output solution
    if LpStatus[problem.status] == 'Optimal':
        for v in problem.variables():
            print(v.name, "=", v.varValue)
    #for k, v in problem.constraints.items():
        #print(k, v)

    print("-------------------------------------------------------")
    print("Amount of Resources Required for Unique Monitoring: {}".format(value(problem.objective)))
    print("Total Number of Nodes to be Uniquely Monitored: {}".format(len(tnodes)))
    print("% Savings: {}".format(float(100 * (len(tnodes) - int(value(problem.objective))) / len(tnodes))))
    print("-------------------------------------------------------")
    print("Time taken = {} seconds".format(time.time() - start))
    print("-------------------------------------------------------")
    #print(G.number_of_edges())

# Entry point
def main():
    model()
if __name__ == '__main__':
    main()